# Modeling: Random Forest and XGBoost

### Importing Libraries

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV, cross_val_score
from tqdm import tqdm_notebook as tqdm
import _pickle as pickle

### Loading in the DataFrame

In [2]:
with open("main_df.pickle",'rb') as fp:
    main_df = pickle.load(fp)

### Preparing Data for Modeling

In [3]:
# Assigning appropriate variables for modeling
X = main_df.drop('price_will_increase?', axis=1)
y = main_df['price_will_increase?']

# Splitting the data 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

## Modeling - Random Forest

In [8]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
# Fitting and training
forest = RandomForestClassifier(n_estimators=100, max_depth=5)
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [13]:
# Accuracy of the RandomForest
score = forest.score(X_test, y_test) * 100
print("Forest Accuracy:", score)

# Testing Area under the Curve
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("AUC:", round(roc_auc, 4))

Forest Accuracy: 65.53518334985134
AUC: 0.5679


In [23]:
y_pred = forest.predict(X_test)
print("Confusion Matrix \n-----------------")
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

Confusion Matrix 
-----------------


Predicted,False,True,All
Actual,,,
False,320,1205,1525
True,186,2325,2511
All,506,3530,4036


In [24]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.63      0.21      0.32      1525
        True       0.66      0.93      0.77      2511

   micro avg       0.66      0.66      0.66      4036
   macro avg       0.65      0.57      0.54      4036
weighted avg       0.65      0.66      0.60      4036



### Improving RandomForest with GridSearchCV

In [28]:
# Using Cross Validation
rf_clf = RandomForestClassifier(n_estimators=100)
rf_cv_score = cross_val_score(rf_clf, X, y, cv=3)
mean_rf_cv_score = np.mean(rf_cv_score)

print("Mean Cross Validation Score:", mean_rf_cv_score*100)

Mean Cross Validation Score: 63.83821829582883


In [29]:
# Setting the Paramters to be tested
rf_param_grid = {'n_estimators': [10,30,100],
                 'criterion': ['gini', 'entropy'],
                 'max_depth': [None, 2, 6, 10],
                 'min_samples_split': [5,10],
                 'min_samples_leaf': [1, 2, 5]}

#### Finding the Best Parameters along with Accuracy

In [31]:
rf_grid_search = GridSearchCV(rf_clf, rf_param_grid, cv=3, return_train_score=True, verbose=10)
rf_grid_search.fit(X, y)

Fitting 3 folds for each of 144 candidates, totalling 432 fits
[CV] criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=10 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=10, score=0.6109128754088612, total=   0.5s
[CV] criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV]  criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=10, score=0.6050557620817844, total=   0.5s
[CV] criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=10 


...

[CV]  criterion=entropy, max_depth=6, min_samples_leaf=1, min_samples_split=10, n_estimators=30, score=0.647636039250669, total=   0.9s
[CV] criterion=entropy, max_depth=6, min_samples_leaf=1, min_samples_split=10, n_estimators=30 
[CV]  criterion=entropy, max_depth=6, min_samples_leaf=1, min_samples_split=10, n_estimators=30, score=0.6422304832713754, total=   1.0s
[CV] criterion=entropy, max_depth=6, min_samples_leaf=1, min_samples_split=10, n_estimators=30 
[CV]  criterion=entropy, max_depth=6, min_samples_leaf=1, min_samples_split=10, n_estimators=30, score=0.6504089219330855, total=   1.0s
[CV] criterion=entropy, max_depth=6, min_samples_leaf=1, min_samples_split=10, n_estimators=100 
[CV]  criterion=entropy, max_depth=6, min_samples_leaf=1, min_samples_split=10, n_estimators=100, score=0.6500148676776687, total=   3.5s
[CV] criterion=entropy, max_depth=6, min_samples_leaf=1, min_samples_split=10, n_estimators=100 
[CV]  criterion=entropy, max_depth=6, min_samples_leaf=1, min_samp

[CV]  criterion=entropy, max_depth=6, min_samples_leaf=5, min_samples_split=10, n_estimators=30, score=0.6504608980077312, total=   0.9s
[CV] criterion=entropy, max_depth=6, min_samples_leaf=5, min_samples_split=10, n_estimators=30 
[CV]  criterion=entropy, max_depth=6, min_samples_leaf=5, min_samples_split=10, n_estimators=30, score=0.6449070631970261, total=   0.9s
[CV] criterion=entropy, max_depth=6, min_samples_leaf=5, min_samples_split=10, n_estimators=30 
[CV]  criterion=entropy, max_depth=6, min_samples_leaf=5, min_samples_split=10, n_estimators=30, score=0.6492193308550186, total=   0.9s
[CV] criterion=entropy, max_depth=6, min_samples_leaf=5, min_samples_split=10, n_estimators=100 
[CV]  criterion=entropy, max_depth=6, min_samples_leaf=5, min_samples_split=10, n_estimators=100, score=0.648528099910794, total=   2.9s
[CV] criterion=entropy, max_depth=6, min_samples_leaf=5, min_samples_split=10, n_estimators=100 
[CV]  criterion=entropy, max_depth=6, min_samples_leaf=5, min_samp

[CV]  criterion=entropy, max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=10, score=0.6408921933085502, total=   0.4s
[CV] criterion=entropy, max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=30 
[CV]  criterion=entropy, max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=30, score=0.6467439785905441, total=   1.3s
[CV] criterion=entropy, max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=30 
[CV]  criterion=entropy, max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=30, score=0.644460966542751, total=   1.3s
[CV] criterion=entropy, max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=30 
[CV]  criterion=entropy, max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=30, score=0.64817843866171, total=   1.3s
[CV] criterion=entropy, max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=100 
[CV]  criterion=entropy, max_depth=10, min_samples_leaf=2, min

[Parallel(n_jobs=1)]: Done 432 out of 432 | elapsed: 12.0min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [10, 30, 100], 'criterion': ['gini', 'entropy'], 'max_depth': [None, 2, 6, 10], 'min_samples_split': [5, 10], 'min_samples_leaf': [1, 2, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [32]:
print(f"Testing Accuracy: {rf_grid_search.best_score_*100}")
print(f"Optimal Parameters: {rf_grid_search.best_params_}")

Testing Accuracy: 64.9534099920698
Optimal Parameters: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 5, 'n_estimators': 100}


#### Using the RandomForest parameters from GridSearch

In [33]:
# Fitting and training
forest = RandomForestClassifier(n_estimators=100, max_depth=10, criterion='entropy', min_samples_leaf=5, min_samples_split=5)
forest.fit(X_train, y_train)

# Accuracy of the RandomForest
score = forest.score(X_test, y_test) * 100
print("Forest Accuracy:", score)

# Testing Area under the Curve
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("AUC:", round(roc_auc, 4))

Forest Accuracy: 65.70862239841428
AUC: 0.5679


In [36]:
# Classification Report
print(classification_report(y_test, y_pred))

# Confusion Matrix
y_pred = forest.predict(X_test)
print("Confusion Matrix \n-------------------------")
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

              precision    recall  f1-score   support

       False       0.63      0.22      0.33      1525
        True       0.66      0.92      0.77      2511

   micro avg       0.66      0.66      0.66      4036
   macro avg       0.65      0.57      0.55      4036
weighted avg       0.65      0.66      0.60      4036

Confusion Matrix 
-------------------------


Predicted,False,True,All
Actual,,,
False,336,1189,1525
True,195,2316,2511
All,531,3505,4036


##### GridSearch slightly improved the model by increasing the accuracy by around 0.2% and AUC remained largely unchanged.

## Modeling - XGBoost

In [4]:
# Preventing error from occuring: XGBoost causes kernel to die.
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier
import xgboost as xgb

In [5]:
x_clf = xgb.XGBClassifier()
x_clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [11]:
# Predictions and accuracy from XGBoost
y_pred = x_clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("XGBoost Accuracy:", acc)

# Area under the curve
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("AUC:", round(roc_auc, 4))

XGBoost Accuracy: 0.6546085232903865
AUC: 0.5705


In [12]:
# Classification Report
print(classification_report(y_test, y_pred))

# Confusion Matrix
y_pred = x_clf.predict(X_test)
print("Confusion Matrix \n-------------------------")
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

              precision    recall  f1-score   support

       False       0.62      0.23      0.33      1525
        True       0.66      0.91      0.77      2511

   micro avg       0.65      0.65      0.65      4036
   macro avg       0.64      0.57      0.55      4036
weighted avg       0.64      0.65      0.60      4036

Confusion Matrix 
-------------------------


Predicted,False,True,All
Actual,,,
False,345,1180,1525
True,214,2297,2511
All,559,3477,4036


### Improving XGBoost with GridSearch

In [14]:
# Using Cross Validation for XGB
x_clf = xgb.XGBClassifier()
x_cv_score = cross_val_score(x_clf, X, y, cv=3)
mean_x_cv_score = np.mean(x_cv_score)

print("Mean Cross Validation Score:", mean_x_cv_score*100)

Mean Cross Validation Score: 64.87907585315966


In [20]:
x_param_grid = {'learning_rate': [1, .1, .01],
                'max_depth': [2, 4, 6],
                'min_child_weight': [4, 10, 20],
                'subsample': [.7, .5, .8],
                'n_estimators': [5,30,100,250]}

In [21]:
x_grid_search = GridSearchCV(x_clf, x_param_grid, cv=3, return_train_score=True, verbose=5)
x_grid_search.fit(X, y)

Fitting 3 folds for each of 324 candidates, totalling 972 fits
[CV] learning_rate=1, max_depth=2, min_child_weight=4, n_estimators=5, subsample=0.7 
[CV]  learning_rate=1, max_depth=2, min_child_weight=4, n_estimators=5, subsample=0.7, score=0.6424323520666072, total=   0.2s
[CV] learning_rate=1, max_depth=2, min_child_weight=4, n_estimators=5, subsample=0.7 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV]  learning_rate=1, max_depth=2, min_child_weight=4, n_estimators=5, subsample=0.7, score=0.641189591078067, total=   0.2s
[CV] learning_rate=1, max_depth=2, min_child_weight=4, n_estimators=5, subsample=0.7 
[CV]  learning_rate=1, max_depth=2, min_child_weight=4, n_estimators=5, subsample=0.7, score=0.6456505576208178, total=   0.1s
[CV] learning_rate=1, max_depth=2, min_child_weight=4, n_estimators=5, subsample=0.5 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.5s remaining:    0.0s


[CV]  learning_rate=1, max_depth=2, min_child_weight=4, n_estimators=5, subsample=0.5, score=0.6419863217365448, total=   0.2s
[CV] learning_rate=1, max_depth=2, min_child_weight=4, n_estimators=5, subsample=0.5 
[CV]  learning_rate=1, max_depth=2, min_child_weight=4, n_estimators=5, subsample=0.5, score=0.6423791821561339, total=   0.1s
[CV] learning_rate=1, max_depth=2, min_child_weight=4, n_estimators=5, subsample=0.5 


...

[CV]  learning_rate=0.01, max_depth=6, min_child_weight=4, n_estimators=30, subsample=0.7, score=0.6457992565055762, total=   1.6s
[CV] learning_rate=0.01, max_depth=6, min_child_weight=4, n_estimators=30, subsample=0.7 
[CV]  learning_rate=0.01, max_depth=6, min_child_weight=4, n_estimators=30, subsample=0.7, score=0.6521933085501859, total=   1.7s
[CV] learning_rate=0.01, max_depth=6, min_child_weight=4, n_estimators=30, subsample=0.5 
[CV]  learning_rate=0.01, max_depth=6, min_child_weight=4, n_estimators=30, subsample=0.5, score=0.648230746357419, total=   1.5s
[CV] learning_rate=0.01, max_depth=6, min_child_weight=4, n_estimators=30, subsample=0.5 
[CV]  learning_rate=0.01, max_depth=6, min_child_weight=4, n_estimators=30, subsample=0.5, score=0.6486245353159852, total=   1.5s
[CV] learning_rate=0.01, max_depth=6, min_child_weight=4, n_estimators=30, subsample=0.5 
[CV]  learning_rate=0.01, max_depth=6, min_child_weight=4, n_estimators=30, subsample=0.5, score=0.6532342007434945, 

[CV]  learning_rate=0.01, max_depth=6, min_child_weight=10, n_estimators=30, subsample=0.7, score=0.6495167286245354, total=   1.6s
[CV] learning_rate=0.01, max_depth=6, min_child_weight=10, n_estimators=30, subsample=0.5 
[CV]  learning_rate=0.01, max_depth=6, min_child_weight=10, n_estimators=30, subsample=0.5, score=0.6486767766874814, total=   1.4s
[CV] learning_rate=0.01, max_depth=6, min_child_weight=10, n_estimators=30, subsample=0.5 
[CV]  learning_rate=0.01, max_depth=6, min_child_weight=10, n_estimators=30, subsample=0.5, score=0.6475836431226766, total=   1.3s
[CV] learning_rate=0.01, max_depth=6, min_child_weight=10, n_estimators=30, subsample=0.5 
[CV]  learning_rate=0.01, max_depth=6, min_child_weight=10, n_estimators=30, subsample=0.5, score=0.652639405204461, total=   1.3s
[CV] learning_rate=0.01, max_depth=6, min_child_weight=10, n_estimators=30, subsample=0.8 
[CV]  learning_rate=0.01, max_depth=6, min_child_weight=10, n_estimators=30, subsample=0.8, score=0.643027059

[CV]  learning_rate=0.01, max_depth=6, min_child_weight=20, n_estimators=30, subsample=0.5, score=0.647933392804044, total=   1.3s
[CV] learning_rate=0.01, max_depth=6, min_child_weight=20, n_estimators=30, subsample=0.5 
[CV]  learning_rate=0.01, max_depth=6, min_child_weight=20, n_estimators=30, subsample=0.5, score=0.64817843866171, total=   1.3s
[CV] learning_rate=0.01, max_depth=6, min_child_weight=20, n_estimators=30, subsample=0.5 
[CV]  learning_rate=0.01, max_depth=6, min_child_weight=20, n_estimators=30, subsample=0.5, score=0.6523420074349442, total=   1.3s
[CV] learning_rate=0.01, max_depth=6, min_child_weight=20, n_estimators=30, subsample=0.8 
[CV]  learning_rate=0.01, max_depth=6, min_child_weight=20, n_estimators=30, subsample=0.8, score=0.6462979482604817, total=   1.6s
[CV] learning_rate=0.01, max_depth=6, min_child_weight=20, n_estimators=30, subsample=0.8 
[CV]  learning_rate=0.01, max_depth=6, min_child_weight=20, n_estimators=30, subsample=0.8, score=0.64594795539

[Parallel(n_jobs=1)]: Done 972 out of 972 | elapsed: 54.1min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=6, min_child_weight=10, missing=None,
       n_estimators=30, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=0.7, verbosity=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'learning_rate': [1, 0.1, 0.01], 'max_depth': [2, 4, 6], 'min_child_weight': [4, 10, 20], 'subsample': [0.7, 0.5, 0.8], 'n_estimators': [5, 30, 100, 250]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=5)

In [22]:
print(f"Testing Accuracy: {x_grid_search.best_score_*100}")
print(f"Optimal Parameters: {x_grid_search.best_params_}")

Testing Accuracy: 65.13679619349723
Optimal Parameters: {'learning_rate': 0.1, 'max_depth': 2, 'min_child_weight': 4, 'n_estimators': 100, 'subsample': 0.5}


In [23]:
# Fitting and training XGB
x_clf = xgb.XGBClassifier(learning_rate=.1, max_depth=2, min_child_weight=4, n_estimators=100, subsample=.5)
x_clf.fit(X_train, y_train)

# Predictions and accuracy from XGBoost
y_pred = x_clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("XGBoost Accuracy:", acc)

# Area under the curve
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("AUC:", round(roc_auc, 4))

XGBoost Accuracy: 0.6585728444003964
AUC: 0.5736


In [19]:
# Classification Report
print(classification_report(y_test, y_pred))

# Confusion Matrix
y_pred = x_clf.predict(X_test)
print("Confusion Matrix \n-------------------------")
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

              precision    recall  f1-score   support

       False       0.62      0.24      0.35      1525
        True       0.66      0.91      0.77      2511

   micro avg       0.66      0.66      0.66      4036
   macro avg       0.64      0.58      0.56      4036
weighted avg       0.65      0.66      0.61      4036

Confusion Matrix 
-------------------------


Predicted,False,True,All
Actual,,,
False,368,1157,1525
True,225,2286,2511
All,593,3443,4036
